In [ ]:
import torch
import pandas
import numpy as np

In [ ]:
import pandas
#dataset taken from https://www.kaggle.com/yashsawarn/wifi-stretgth-for-rooms

def read_dataset(csv_name = 'wifi_localization.txt'):
    """
    Reads a csv dataset
    returns it as a pytorch tensor
    """
    data_frame = pandas.read_table(csv_name, delim_whitespace=True, names=('A', 'B', 'C', 'D','E', 'F', 'G', 'ROOM'),
                       dtype={'A': np.int64, 'B': np.float64, 'C': np.float64, 'D': np.float64,'E': np.float64,'F': np.float64,'G': np.float64,'ROOM': np.float64})

    targets_torch = torch.tensor(data_frame['ROOM'].values)
    dataset_torch = torch.tensor(data_frame.values)

    return dataset_torch
dataset_torch = read_dataset()



In [ ]:
print(dataset_torch)

tensor([[-64., -56., -61.,  ..., -82., -81.,   1.],
        [-68., -57., -61.,  ..., -85., -85.,   1.],
        [-63., -60., -60.,  ..., -85., -84.,   1.],
        ...,
        [-62., -59., -46.,  ..., -87., -88.,   4.],
        [-62., -58., -52.,  ..., -90., -85.,   4.],
        [-59., -50., -45.,  ..., -88., -87.,   4.]], dtype=torch.float64)


In [1]:
class Node_CART:
    def __init__(self, num_classes = 4, ref_CART = None, current_depth = 0):
        """
        Create the node attributes
        param num_classes: K number of classes to classify
        param ref_cart: reference to the tree containing the node
        param current_depth: current depth of the node in the tree
        """
        self.ref_CART = ref_CART
        self.threshold_value = 0
        self.feature_num = 0
        self.node_right = None
        self.node_left = None
        self.data_torch_partition = None
        self.gini = 0
        self.dominant_class = None
        self.accuracy_dominant_class = None
        self.num_classes = num_classes
        self.current_depth = current_depth

    def to_xml(self, current_str = ""):
        """
        Recursive function to write the node content to an xml formatted string
        param current_str : the xml content so far in the whole tree
        return the string with the node content
        """
        str_node = "<node><thresh>" + str(self.threshold_value) + "</thresh>" + "<feature>" + str(self.feature_num) + "</feature><depth>" + str(self.current_depth)+ "</depth>"
        str_node += "<gini>" + str(self.gini) + "</gini>"
        if(self.node_right != None):
            str_left = self.node_right.to_xml(current_str)
            str_node += str_left
        if(self.node_left != None):
            str_right = self.node_left.to_xml(current_str)
            str_node += str_right

        if(self.is_leaf()):
            str_node += "<dominant_class>" + str(self.dominant_class) + "</dominant_class><acc_dominant_class>"  + str(self.accuracy_dominant_class) + "</acc_dominant_class>"
        str_node += "</node>"
        return str_node

    def is_leaf(self):
        """
        Checks whether the node is a leaf
        """
        return (self.node_left == None and self.node_right == None)

    def create_with_children(self, data_torch, current_depth, min_gini = 0.000001):
        """
        Creates a node by selecting the best feature and threshold, and if needed, creating its children
        param data_torch: dataset with the current partition to deal with in the node
        param current_depth: depth counter for the node
        param min_gini: hyperparmeter selected by the user defining the minimum tolerated gini coefficient for a  node
        return the list of selected features so far
        """


        return list_selected_features


def select_best_feature_and_thresh(data_torch, num_classes=2):
    """
    Selecciona el atributo y el umbral que minimizan el Gini ponderado.

    Parámetros
    ----------
    data_torch : torch.Tensor
        Tensor con datos donde la última columna es la etiqueta de clase.
    num_classes : int, por defecto 2
        Número de clases posibles.

    Retorna
    -------
    best_feature : int
        Índice de la característica que produce la menor impureza de Gini.
    best_thresh : float
        Valor umbral para dividir en esa característica.
    best_gini : float
        Impureza de Gini ponderada para esta división óptima.

    Detalles de implementación
    --------------------------
    - Se itera solo sobre atributos y umbrales posibles.
    - Se usan máscaras booleanas para dividir los datos:
        left_mask = data[:, f] <= t
        right_mask = ~left_mask
    - Gini ponderado: (nL/n)*GL + (nR/n)*GR
    - Se busca el mínimo sobre todas las divisiones posibles.
    """
    n_samples, n_features_plus_label = data_torch.shape
    n_features = n_features_plus_label - 1

    best_feature, best_thresh, best_gini = None, None, float('inf')

    # Iterar sobre cada atributo (feature)
    for f in range(n_features):
        # Valores únicos como candidatos de umbral
        values = torch.unique(data_torch[:, f])

        for thresh in values:
            # Crear máscaras booleanas sin bucles
            left_mask = data_torch[:, f] <= thresh
            right_mask = ~left_mask

            left_data = data_torch[left_mask]
            right_data = data_torch[right_mask]

            # Calcular gini de cada lado
            gini_left = calculate_gini(left_data, num_classes)
            gini_right = calculate_gini(right_data, num_classes)

            # Calcular gini ponderado
            n_left, n_right = left_data.shape[0], right_data.shape[0]
            gini_split = (n_left * gini_left + n_right * gini_right) / n_samples

            # Verificar si es la mejor división
            if gini_split < best_gini:
                best_gini = gini_split
                best_feature = f
                best_thresh = thresh.item()

    return best_feature, best_thresh, best_gini


    def calculate_gini(data_partition_torch, num_classes=2):
      """
      Calcula el coeficiente de Gini para una partición de datos.

      Parámetros
      ----------
      data_partition_torch : torch.Tensor
          Tensor donde la última columna contiene las etiquetas de clase.
      num_classes : int, por defecto 2
          Número de clases posibles en el conjunto de datos.

      Retorna
      -------
      gini : float
          Impureza de Gini para la partición dada.

      Detalles de implementación
      --------------------------
      - torch.bincount: cuenta el número de ocurrencias de cada etiqueta.
      - torch.sum: suma los elementos de un tensor.
      - Se evita el uso de bucles de Python: todo está vectorizado.
      - Fórmula: G = 1 - Σ(p_k^2), donde p_k = frecuencia relativa de la clase k.
      """
      if data_partition_torch.numel() == 0:
          # Si la partición está vacía, por definición no hay impureza.
          return 0.0

      # Extraer etiquetas (suponemos que la última columna son las etiquetas)
      labels = data_partition_torch[:, -1].long()

      # Contar ocurrencias por clase (minlength garantiza tamaño fijo del vector)
      counts = torch.bincount(labels, minlength=num_classes)

      # Convertir a probabilidades dividiendo entre el total
      probs = counts.float() / counts.sum()

      # Calcular Gini: 1 - suma de cuadrados de probabilidades
      gini = 1.0 - torch.sum(probs ** 2)
      return gini.item()

    def calculate_gini(self, data_partition_torch, num_classes = 4):
        """
        Calculates the gini coefficient for a given partition with the given number of classes
        param data_partition_torch: current dataset partition as a tensor
        param num_classes: K number of classes to discriminate from
        returns the calculated gini coefficient
        """
        def gini_for_subset(subset):
            if subset.shape[0] == 0:
                return 0.0
            gini = 1.0
            total = subset.shape[0]
            for cls in range(num_classes):
                count = torch.sum(subset[:, -1] == cls).item()
                prob = count / total
                gini -= prob ** 2
            return gini
        gini = gini_for_subset(data_partition_torch)
        return gini



    def evaluate_node(self, input_torch):
        """
        Evaluates an input observation within the node.
        If is not a leaf node, send it to the corresponding node
        return predicted label
        """
        feature_val_input = input_torch[self.feature_num]
        if(self.is_leaf()):
            return self.dominant_class
        else:
            if(feature_val_input < self.threshold_value):
                return self.node_left.evaluate_node(input_torch)
            else:
                return self.node_right.evaluate_node(input_torch)

In [ ]:
dataset_example = torch.tensor([[3, 22.0, 7.2, 0], [1, 38, 71.3, 0], [3, 26, 7.9, 1], [1, 35, 53.1, 0]])
print("dataset_example \n", dataset_example)


dataset_example 
 tensor([[ 3.0000, 22.0000,  7.2000,  0.0000],
        [ 1.0000, 38.0000, 71.3000,  0.0000],
        [ 3.0000, 26.0000,  7.9000,  1.0000],
        [ 1.0000, 35.0000, 53.1000,  0.0000]])


In [ ]:
root_node = Node_CART()
root_node.data_torch_partition = dataset_example

print("root node \n ", root_node.data_torch_partition)
xml_root_node = root_node.to_xml()

print("xml_root_node \n", xml_root_node)

root node 
  tensor([[ 3.0000, 22.0000,  7.2000,  0.0000],
        [ 1.0000, 38.0000, 71.3000,  0.0000],
        [ 3.0000, 26.0000,  7.9000,  1.0000],
        [ 1.0000, 35.0000, 53.1000,  0.0000]])
xml_root_node 
 <node><thresh>0</thresh><feature>0</feature><depth>0</depth><gini>0</gini><dominant_class>None</dominant_class><acc_dominant_class>None</acc_dominant_class></node>


In [ ]:
root_node.threshold_value = 3
root_node.feature_num = 0

#indices of left and right partitions
left_idxs = dataset_example[:, root_node.feature_num] < root_node.threshold_value
right_idxs = dataset_example[:, root_node.feature_num] >= root_node.threshold_value
#data partitions
dataset_partition_left = dataset_example[left_idxs]
dataset_partition_right = dataset_example[right_idxs]

print("dataset_partition_left \n", dataset_partition_left)
print("dataset_partition_right \n", dataset_partition_right)
#create left child
left_child = Node_CART(current_depth = 1)
left_child.data_torch_partition = dataset_partition_left
root_node.node_left = left_child
#create right child
right_child = Node_CART(current_depth = 1)
right_child.data_torch_partition = dataset_partition_right
root_node.node_right = right_child
#write xml example
root_node.ref_CART = root_node
xml_string = root_node.to_xml()

#print(xml_string)
file = open("example1.xml", "a")
file.write(xml_string)
file.close()


dataset_partition_left 
 tensor([[ 1.0000, 38.0000, 71.3000,  0.0000],
        [ 1.0000, 35.0000, 53.1000,  0.0000]])
dataset_partition_right 
 tensor([[ 3.0000, 22.0000,  7.2000,  0.0000],
        [ 3.0000, 26.0000,  7.9000,  1.0000]])


In [ ]:
print(xml_string)

<node><thresh>3</thresh><feature>0</feature><depth>0</depth><gini>0</gini><node><thresh>0</thresh><feature>0</feature><depth>1</depth><gini>0</gini><dominant_class>None</dominant_class><acc_dominant_class>None</acc_dominant_class></node><node><thresh>0</thresh><feature>0</feature><depth>1</depth><gini>0</gini><dominant_class>None</dominant_class><acc_dominant_class>None</acc_dominant_class></node></node>


In [ ]:
def calculate_gini(node_left, node_right):
  size_left = node_left.data_torch_partition.shape[0]
  size_right = node_right.data_torch_partition.shape[0]
  size_total = size_left + size_right
  gini_left = 1 - (((node_left.data_torch_partition[:, 3] == 0).sum().item() / size_left)**2 + ((node_left.data_torch_partition[:, 3] == 1).sum().item()/size_left)**2)
  gini_right = 1 - (((node_right.data_torch_partition[:, 3] == 0).sum().item() / size_right)**2 + ((node_right.data_torch_partition[:, 3] == 1).sum().item()/size_right)**2)
  gini_total = (size_left / size_total * gini_left) + (size_right / size_total * gini_right)
  print("gini_left: ", gini_left, "gini_right: ", gini_right, "gini_total: ", gini_total)
  return gini_total
gini_total = calculate_gini(left_child, right_child)
print("gini_total ", gini_total)

gini_left:  0.4444444444444444 gini_right:  0.0 gini_total:  0.3333333333333333
gini_total  0.3333333333333333


In [ ]:
# Prueba unitaria 1: Gini para un solo grupo homogéneo (impureza debe ser 0)
def test_gini_homogeneous():
    node = Node_CART(num_classes=2)
    # Todos los elementos son de la clase 0
    data = torch.tensor([[1, 2, 0], [2, 3, 0], [3, 4, 0]], dtype=torch.float32)
    gini = node.calculate_gini(data, num_classes=2)
    print('Test 1 - Gini homogéneo:', gini)
    assert abs(gini - 0.0) < 1e-6, f"Esperado 0.0, obtenido {gini}"

# Prueba unitaria 2: Gini para dos clases balanceadas (impureza máxima)
def test_gini_balanced():
    node = Node_CART(num_classes=2)
    # Mitad clase 0, mitad clase 1
    data = torch.tensor([[1, 2, 0], [2, 3, 1], [3, 4, 0], [4, 5, 1]], dtype=torch.float32)
    gini = node.calculate_gini(data, num_classes=2)
    print('Test 2 - Gini balanceado:', gini)
    assert abs(gini - 0.5) < 1e-6, f"Esperado 0.5, obtenido {gini}"

# Ejecutar pruebas
if __name__ == "__main__":
    test_gini_homogeneous()
    test_gini_balanced()